# Problem1

Using Book1 dataset program 3 different waysof crossvalidations (holdout, K = 10 fold cross validation, and leave-one-out cross validation)for 3different types of models: linear regression,decision treeand support vector machinerepressorwith “linear”kernel and your choice of other parameters(do not worry about hyper parametersaccept the model as a “black box”).

What can you conclude from cross validations about these 3 models?

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimization

In [2]:
#load data
data = pd.read_excel('Book1.xlsx', header = None)
data = np.transpose(data.as_matrix())

In [3]:
#small function to make MSE as output of linaer regression
def lr_mse(x_train,y_train, x_test, y_test):
    def func(x, a, b): return a + b*x
    x0 = np.array([0.0,0.0])
    optpar = optimization.curve_fit(func,x_train,y_train,x0)
    a = optpar[0][0]
    b = optpar[0][1]
    
    #predict
    s = 0
    if np.size(x_test) == 1:
        s = (y_test - (a+b*x_test))**2
    else:
        for n in range(np.size(x_test)):
            s = s + (y_test[n] - (a+b*x_test[n]))**2
    return s/np.size(x_test)

#small function to make MSE as output of decision tree
#depth is the max_depth for decision tree
def dt_mse(x_train,y_train, x_test, y_test, depth):
    x_train = x_train.reshape(-1,1)
    y_train = np.ravel(y_train)
    x_test = x_test.reshape(-1,1)
    y_test = np.ravel(y_test)
    
    from sklearn.tree import DecisionTreeRegressor
    
    tree_reg = DecisionTreeRegressor(max_depth = depth)
    tree_reg.fit(x_train,y_train)
    
    # Predict
    y_predict = tree_reg.predict(x_test)
    s = 0
    if np.size(x_test) ==  1:
        s = (y_predict - y_test)**2
    else:
        for n in range(np.size(x_test)):
            s = s + (y_predict[n] - y_test[n])**2
    return s/np.size(x_test)

#small function to make MSE as output of SVM
def svm_mse(x_train, y_train, x_test, y_test, c, eps):
    x_train = x_train.reshape(-1,1)
    y_train = np.ravel(y_train)
    x_test = x_test.reshape(-1,1)
    y_test = np.ravel(y_test)
    
    from sklearn.svm import SVR
    
    svm_linear_reg = SVR(kernel = "linear", C = c, epsilon = eps,cache_size = 2000)
    svm_linear_reg.fit(x_train, y_train)
    
    #predict
    y_predict = svm_linear_reg.predict(x_test)
    s = 0
    if np.size(x_test) == 1:
        s = (y_predict - y_test)**2
    else:
        for n in range(np.size(x_test)):
            s = s + (y_predict[n] - y_test[n])**2
    return s/np.size(x_test)


In [4]:
#holdout 20% 
matr = np.arange(len(data[0]))
selection1 = np.random.permutation(matr)
num = int(0.8*len(data[0]))
selection = [selection1[i] for i in range(0,num)]  
selection = np.array(selection)

dataTrain_ho = np.array([i for i in np.transpose(data) if i[0] in selection])
dataTest_ho = np.array([i for i in np.transpose(data) if i[0] not in selection])
dataTrain_ho = np.transpose(dataTrain_ho)
dataTest_ho = np.transpose(dataTest_ho)

mse_lr_handout = lr_mse(dataTrain_ho[1], dataTrain_ho[2], dataTest_ho[1], dataTest_ho[2])
mse_dt_handout = dt_mse(dataTrain_ho[1], dataTrain_ho[2], dataTest_ho[1], dataTest_ho[2], 10)
mse_svm_handout = svm_mse(dataTrain_ho[1], dataTrain_ho[2], dataTest_ho[1], dataTest_ho[2],1,0.5)

In [5]:
print('the mse generated by the linear regression is:', mse_lr_handout, 
      'the mse generated by the decision tree is:', mse_dt_handout, 'the mse generated by svm is:', mse_svm_handout)

the mse generated by the linear regression is: 101.455985291 the mse generated by the decision tree is: 108.095308967 the mse generated by svm is: 101.472671497


In [6]:
#k fold with k = 10
spl = np.array_split(selection1,10)
dataTest_fold = list(range(10))
dataTrain_fold = list(range(10))
for i in range(10):
    selection_fold = spl[i]
    dataTest_fold_copy = np.array([m for m in np.transpose(data) if m[0] in selection_fold])
    dataTrain_fold_copy = np.array([m for m in np.transpose(data) if m[0] not in selection_fold])
    dataTrain_fold[i] = np.transpose(dataTrain_fold_copy)
    dataTest_fold[i] = np.transpose(dataTest_fold_copy)

mse_lr_kfold = 0
mse_dt_kfold = 0
mse_svm_kfold = 0

for i in range(10):
    mse_lr_kfold =  mse_lr_kfold +  lr_mse(dataTrain_fold[i][1], dataTrain_fold[i][2], 
                                           dataTest_fold[i][1], dataTest_fold[i][2])
    mse_dt_kfold = mse_dt_kfold + dt_mse(dataTrain_fold[i][1], dataTrain_fold[i][2], 
                                           dataTest_fold[i][1], dataTest_fold[i][2],10)
    mse_svm_kfold =  mse_svm_kfold +  svm_mse(dataTrain_fold[i][1], dataTrain_fold[i][2], 
                                           dataTest_fold[i][1], dataTest_fold[i][2],1,0.5)
mse_lr_kfold = mse_lr_kfold/10
mse_dt_kfold = mse_dt_kfold/10
mse_svm_kfold = mse_svm_kfold/10

In [7]:
print("the mse generated by the linear regression is:", mse_lr_kfold,", the mse generated by the decision tree is:",
     mse_dt_kfold, ", the mse generated by svm is:", mse_svm_kfold)

the mse generated by the linear regression is: 101.605449567 , the mse generated by the decision tree is: 106.687544091 , the mse generated by svm is: 101.607921285


In [8]:
#it seems like hard to train svm with a large data set in a short time
#for the leave-one-out method, let's turn it into a smaller set

#pick up 2000 data randomly from the original dataset, and train with leave-one-out cross vaildation
data_for_leave = selection1
selection = [selection1[i] for i in range(0,2000)] 
data_for_leave = np.array([i for i in np.transpose(data) if i[0] in selection])

mse_lr_leave_1 = 0
mse_dt_leave_1 = 0
mse_svm_leave_1 = 0

#leave-one-out 
for i in range(len(data_for_leave)):
    dataTest_leave_1_out = data_for_leave[i]
    dataTrain_leave_1_out = np.delete(data_for_leave,i,0)
    mse_lr_leave_1 = mse_lr_leave_1 + lr_mse(dataTrain_leave_1_out[:,1],dataTrain_leave_1_out[:,2],
                                             dataTest_leave_1_out[1],dataTest_leave_1_out[2])
    mse_dt_leave_1 = mse_lr_leave_1 + dt_mse(dataTrain_leave_1_out[:,1],dataTrain_leave_1_out[:,2],
                                             dataTest_leave_1_out[1],dataTest_leave_1_out[2],10)
    mse_svm_leave_1 = mse_svm_leave_1 + svm_mse(dataTrain_leave_1_out[:,1],dataTrain_leave_1_out[:,2],
                                             dataTest_leave_1_out[1],dataTest_leave_1_out[2],1,0.5)

mse_lr_leave_1 = mse_lr_leave_1/len(data_for_leave)
mse_dt_leave_1 = mse_dt_leave_1/len(data_for_leave)
mse_svm_leave_1 = mse_svm_leave_1/len(data_for_leave)   

In [9]:
print("the mse generated by the linear regression is:", mse_lr_leave_1,", the mse generated by the decision tree is:",
     float(mse_dt_leave_1), ", the mse generated by svm is:", float(mse_svm_leave_1))

the mse generated by the linear regression is: 101.950282604 , the mse generated by the decision tree is: 102.05967137828733 , the mse generated by svm is: 101.8790100093584


As the result shows, the SVM and linear regression method are more suitbale for the data than the decision tree.


However, there is not much different for the MSE that generated from this three vaildation.
I could say that, the result of leave-one-out is most trustable, it won't change with the same dataset. However, it take a great time to complete the cross vaildation. This method is not ideal with huge data or complex model. K-fold is better than simply holdout method, the result of holdout method could change with different pieces of data it picks out, while with k-fold, i can generated a mean of MSE with different pieces of data, and k-fold won't take much time like leave-one-out.